# Exploring Walrus HTTP API

In [1]:
#! pip install restmagic
#! docker-compose up -d walrus

%load_ext restmagic
%rest_root --timeout 30 http://127.0.0.1:31415

import random
import string

Requests defaults are set.


In [2]:
text = ''.join(random.choice(string.ascii_lowercase) for i in range(128))
text

'yltvgypbxcjmbtntovzsukefmedztcbyzlxgepvozopyxczbnxngsfkadpbhkngpjohkrlflyuxwmjigukrceixujwdllxfmvyuxyqnrmmzddafatxxugmqzqyitqcgc'

### Upload a new data

In [3]:
%%rest PUT v1/store

$text

{
  "newlyCreated": {
    "blobObject": {
      "id": "0x8920bba31a92e5227ca0e0fd9d32d6913a7c89fbf93beebbc6aadb189df61cb2",
      "storedEpoch": 0,
      "blobId": "CDNAVMBlr_vca0TX7ctrUehCb9bIiy1uO1S1J0OtFaM",
      "size": 128,
      "erasureCodeType": "RedStuff",
      "certifiedEpoch": 0,
      "storage": {
        "id": "0x06964d68e6e07ddfc27c3c905ade5cd2a4dae2c703069743fb147bc1f6ac20bd",
        "startEpoch": 0,
        "endEpoch": 1,
        "storageSize": 65023000
      }
    },
    "encodedSize": 65023000,
    "cost": 3175000
  }
}

<Response [200]>

### Repeat request with same data

In [4]:
%%rest --timeout 30 PUT http://127.0.0.1:31415/v1/store

$text

{
  "alreadyCertified": {
    "blobId": "CDNAVMBlr_vca0TX7ctrUehCb9bIiy1uO1S1J0OtFaM",
    "event": {
      "txDigest": "CGdkdbXrgboBSvELG8WRnMBLAPRxvXK41XP2QL2PMeU9",
      "eventSeq": "0"
    },
    "endEpoch": 1
  }
}

<Response [200]>

In [5]:
r = _
blob_id = r.json()["alreadyCertified"]["blobId"]
blob_id

'CDNAVMBlr_vca0TX7ctrUehCb9bIiy1uO1S1J0OtFaM'

### Read the data

In [6]:
%rest GET /v1/$blob_id

yltvgypbxcjmbtntovzsukefmedztcbyzlxgepvozopyxczbnxngsfkadpbhkngpjohkrlflyuxwmjigukrceixujwdllxfmvyuxyqnrmmzddafatxxugmqzqyitqcgc

<Response [200]>

In [7]:
assert _.text == text